### Set up

In [71]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.optimize import minimize
from scipy.stats import shapiro

In [2]:
df = pd.read_csv('winequalityred.csv')

In [3]:
col_names = [i.replace("\"","").replace('\'','') for i in df.columns[0].split(';')]

In [4]:
df = pd.DataFrame(df.iloc[:,0].str.split(';').tolist(),columns = col_names)
df = df.apply(pd.to_numeric)

In [5]:
X = df.drop('quality', axis = 1)

In [6]:
y = df['quality']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=.2, 
                                                    random_state=0)

### Regression equations and functions

\begin{equation}
 y =  \sum_{j=1}^{p} X_{j}*B_{j} 
\end{equation}
\begin{equation}
RSS =  \sum_{i=1}^{N} (y_{i} - B_{o} - \sum_{j=1}^{p} x_{ij}*B_{j})^2
\end{equation}


In [8]:
def lm_predict(X, beta):
    return X.apply(lambda z: np.dot(z, beta), axis = 1)

In [9]:
def RSS(beta, X, y):
    return ((y - lm_predict(X, beta))**2).sum()

### Optimizing the Model

In [200]:
res = minimize(RSS, x0= np.random.normal(0,1,X_train.shape[1]), args = (X_train, y_train))

In [201]:
beta_hat = res.x

### Qualitative Results

In [81]:
beta_hat[(-beta_hat).argsort()]

array([ 4.19185222e+00,  8.74711336e-01,  3.00696788e-01,  1.31235567e-02,
        8.44069623e-03,  3.02578751e-03, -2.87979385e-03, -1.79425350e-01,
       -4.27624486e-01, -1.17657060e+00, -1.93704449e+00])

In [79]:
###largest to smallest
X.columns[(-beta_hat).argsort()]

Index(['density', 'sulphates', 'alcohol', 'residual sugar', 'fixed acidity',
       'free sulfur dioxide', 'total sulfur dioxide', 'citric acid', 'pH',
       'volatile acidity', 'chlorides'],
      dtype='object')

In [66]:
np.mean(X['total sulfur dioxide'])

46.46779237023139

In [67]:
np.mean(X['density'])

0.9967466791744833

Features that are positively correlated to good quality are density, sulphates, alcohol, residual sugar, fixed acidity, free sulfur dioxide. 

Features that are negatively correlated to good quality are citric acid, pH, volaitile acitidy and chlorides. 

Scale of the features can have a large impact to the size of the coefficients. For example, density is averaged at .9967. So if we were to increase it by just one increment that would be double the average. On the other hand, total sulfur dioxide's average is at 46.46 so increment of one is not a proportionally large change.

### RSS in Test Data

In [70]:
RSS(beta_hat, X_train, y_train)

545.5359871242772

In [68]:
RSS(beta_hat, X_test, y_test)

122.42627202457935

In [78]:
shapiro((y_test-lm_predict(X_test, beta_hat)))

(0.9787293672561646, 0.00011085698497481644)

Our model fits relatively well in test data than in the train data. Our RSS is much smaller in the test data at 122.42 as opposed to 545, which suggests that the model generalizes well. However, when we run the shaprio wilks test on the residuals, we see that we reject the null hypothesis that the residuals are normally distributed, which is a strong assumption of linear models. Therefore, there is a reason to believe that linear regression does not fit very well in this data and some of the assumptions are violated.

### Exploring Different Initial $B_o$ Values

In [90]:
res_zeros = minimize(RSS, x0= np.zeros(X_train.shape[1]), args = (X_train, y_train))

In [91]:
res_zeros.fun

545.5359871239784

In [92]:
res_zeros.x

array([ 8.44077327e-03, -1.17657070e+00, -1.79425566e-01,  1.31235577e-02,
       -1.93704398e+00,  3.02578604e-03, -2.87979257e-03,  4.19184864e+00,
       -4.27623622e-01,  8.74711330e-01,  3.00696798e-01])

In [93]:
beta_hat

array([ 8.44075295e-03, -1.17657063e+00, -1.79425396e-01,  1.31235602e-02,
       -1.93704483e+00,  3.02578642e-03, -2.87979293e-03,  4.19184915e+00,
       -4.27623733e-01,  8.74711508e-01,  3.00696789e-01])

In [94]:
res_large = minimize(RSS, x0= np.random.normal(0,1000,X_train.shape[1]), args = (X_train, y_train))

In [95]:
res_large.fun

545.5359871239339

In [96]:
res_large.x

array([ 8.44077651e-03, -1.17657067e+00, -1.79425602e-01,  1.31235596e-02,
       -1.93704369e+00,  3.02578646e-03, -2.87979236e-03,  4.19184913e+00,
       -4.27623845e-01,  8.74711099e-01,  3.00696828e-01])

Solver returns the same results (both RSS and the Beta_hat) for a very small initial values(zeros) and very large initial values(normal distribution with a large standard deviation)

### Trying Different Solver Choices

In [104]:
res_nelder_mead = minimize(RSS, x0= np.random.normal(0,1,X_train.shape[1]), args = (X_train, y_train), method ='Nelder-Mead')

In [107]:
res_nelder_mead.fun

663.0525707160349

In [108]:
res_nelder_mead.x

array([ 1.85541334e-01, -1.24567257e+00, -1.22959360e+00, -8.11333468e-03,
       -1.62813603e-01,  4.46845091e-03,  4.81017690e-04, -2.12362156e-01,
       -3.63906195e-01,  1.25649638e+00,  5.38135537e-01])

In [105]:
res_powell = minimize(RSS, x0= np.random.normal(0,1,X_train.shape[1]), args = (X_train, y_train), method ='Powell')

In [109]:
res_powell.fun

548.0188468930614

In [110]:
res_powell.x

array([ 2.29905277e-02, -1.17590807e+00, -2.58901118e-01,  6.18750697e-03,
       -1.52244470e+00,  2.86498584e-03, -2.35266273e-03,  3.52686126e+00,
       -3.89837390e-01,  7.87289495e-01,  3.44479569e-01])

In [113]:
###original solution
res.fun

545.5359871241537

In [114]:
res.x

array([ 8.44073270e-03, -1.17657103e+00, -1.79425880e-01,  1.31235568e-02,
       -1.93704189e+00,  3.02578688e-03, -2.87979344e-03,  4.19185168e+00,
       -4.27624410e-01,  8.74711011e-01,  3.00696820e-01])

Choice of the solver changes the results. In this case, both solvers were less performant than the original solver(one of BFGS, L-BFGS, SLSQP). However, Powell solver was much closer and the coefficients all have the same direction as the original solution.

### Regularization

\begin{equation}
argmin  \sum_{i=1}^{N} (y_{i} - B_{o} - \sum_{j=1}^{p} x_{ij}*B_{j})^2 + \lambda*\sum_{j=1}^{p}B_{j}^2
\end{equation}

In [191]:
def ridge(beta, X, y, lambda_):
    return ((y - lm_predict(X, beta))**2).sum() + (lambda_)*(beta**2).sum()

In [192]:
lambda_=0.01

In [193]:
res_ridge = minimize(ridge, x0= np.random.normal(0,1,X_train.shape[1]), args = (X_train, y_train, lambda_))

In [198]:
res_ridge.x

array([ 9.41319740e-03, -1.17714032e+00, -1.80838206e-01,  1.31010424e-02,
       -1.91843956e+00,  3.00833749e-03, -2.86312350e-03,  4.13865378e+00,
       -4.15201443e-01,  8.73960463e-01,  3.00973416e-01])

In [199]:
res_ridge.fun

545.7711323715528

In [203]:
res.x

array([ 8.44068992e-03, -1.17657061e+00, -1.79425332e-01,  1.31235580e-02,
       -1.93704441e+00,  3.02578768e-03, -2.87979400e-03,  4.19185249e+00,
       -4.27624549e-01,  8.74711352e-01,  3.00696785e-01])

In [202]:
res.fun

545.5359871243074

In [205]:
RSS(res_ridge.x, X_test,y_test)

122.46057009754884

There is very little change in the results, RSS and coefficients are still very similar. Also RSS in the test results are similar

In [222]:
output = dict()
for lambda_ in [0,.5,1,5,10]:
    res_ridge = minimize(ridge, x0= np.random.normal(0,1,X_train.shape[1]), args = (X_train, y_train, lambda_))
    output[lambda_] = [res_ridge.x, res_ridge.fun,RSS(res_ridge.x, X_test,y_test) ]    

In [223]:
##Test RSS
output[0][2], output[.5][2], output[1][2], output[5][2], output[10][2]

(122.4262801337932,
 123.99003656137126,
 124.9638166237936,
 126.7858957789673,
 127.1108642626774)

In [224]:
output[10][0]

array([ 0.07196888, -0.91380457, -0.07295217,  0.00661469, -0.22748663,
        0.00307556, -0.00198147,  0.38902101,  0.38230478,  0.64411446,
        0.3377126 ])

In [226]:
res.x

array([ 8.44068992e-03, -1.17657061e+00, -1.79425332e-01,  1.31235580e-02,
       -1.93704441e+00,  3.02578768e-03, -2.87979400e-03,  4.19185249e+00,
       -4.27624549e-01,  8.74711352e-01,  3.00696785e-01])

Output suggests that no L2 regularization minimizes the RSS in the test data. We can additionally observe that with high penalty, L2 at 10, We see much much smaller coefficients relative to the original

In [230]:
def lasso(beta, X, y, lambda_):
    return ((y - lm_predict(X, beta))**2).sum() + (lambda_)*(np.abs(beta)).sum()

In [240]:
output = dict()
for lambda_ in [0,.5,1,5,10,1000]:
    res_lasso = minimize(lasso, x0= np.random.normal(0,1,X_train.shape[1]), args = (X_train, y_train, lambda_))
    output[lambda_] = [res_lasso.x, res_lasso.fun,RSS(res_lasso.x, X_test,y_test) ]    

C:\Users\James\Anaconda3\lib\site-packages\scipy\optimize\optimize.py:1013: RuntimeWarning: divide by zero encountered in double_scalars
  rhok = 1.0 / (numpy.dot(yk, sk))


In [244]:
##Test RSS
output[0][2], output[.5][2], output[1][2], output[5][2], output[10][2], output[1000][2]

(122.42627139599216,
 122.68427642359296,
 123.01754973961307,
 125.36196061061278,
 128.59292679448922,
 140.26041693857604)

In [254]:
output[1000][0]

array([ 8.37079958e-02, -4.60123766e-09, -1.36890025e-09, -3.65388234e-09,
       -6.92654235e-09,  4.24027443e-03, -2.62766091e-09, -1.19732848e-08,
        2.54860025e-09, -2.76771624e-09,  4.64152887e-01])

Output suggests that no L1 regularization minimizes the RSS in the test data. We can additionally observe that with high penalty, L1 at 1000, We see some coefficients go to almost zero

Magnitude can absolute affect the regularization. For example, much smaller scaled feature leads to higher valued coefficient. When we do an L2 regularization on such features, they will be penalized higher because of the larger magnitude of the coefficient. This is in a sense unfair to the smaller scale features and could lead to less effective models